#### ZADANIE 1

###### Do zadania załączone są 3 pliki CSV:
    • Zamówienia
    • Cena bazowa
    • Wysłane
    W pliku zamówienia są informacje o zamówieniach klientów w wybranym okresie. 
    W pliku znajdują się kolumny: 
    • order_id – unikalny numer zamówienia
    • client_id – unikalny numer klienta
    • order_date – data złożenia zamówienia
    • kupon – informacja o tym, czy podczas składania zamówienia klient użył kuponu rabatowego obniżającego cenę bazową; wartość kuponu to procent zniżki od ceny bazowej
    W pliku Cena bazowa znajdują się informacje o cenie bazowej każdego z produktów.
    W pliku Wysłane znajdują się informacje o produktach wysłanych ostatecznie do każdego zamówienia. W jednym zamówieniu mogło się znaleźć więcej niż 1 produkt. 
    Na podstawie załączonych danych odpowiedz na pytania:
    1) W jakim dniu roku klienci złożyli najwięcej zamówień?
    2) Ilu klientów skorzystało z kuponu rabatowego w trakcie zakupów?
    3) Ilu klientów zrobiło w analizowanym okresie więcej niż jedno zamówienie?
    4) Który z produktów cieszył się największym powodzeniem? Ilu klientów kupiło go ze zniżką?
    5) Który produkt był najczęściej kupowany ze zniżką?
    6) Jaka była końcowa wartość wszystkich zamówień w badanym okresie?
    Proszę o przesłanie odpowiedzi na pytania i kodu R/ Python, który posłużył do ich wygenerowania.

In [1]:
import pandas as pd
import numpy as np
#import datasets
df_zamow = pd.read_csv('zamowienia.csv', sep = ';')
df_cena = pd.read_csv('cena bazowa.csv', sep =';')
df_wyslane = pd.read_csv('wyslane.csv', sep = ';')

In [2]:
df_zamow.head()

,order_id,client_id,order_date,KUPON
0,1462414,523933015,29.06.2019,NaN
1,1538139,814107372,15.08.2019,NaN
2,1838871,713139458,19.08.2019,NaN
3,1896544,123641536,05.05.2019,15%
4,1943852,602252913,30.07.2019,NaN


In [3]:
#fill kupon column with 0 where NaN
df_zamow['KUPON'] = df_zamow['KUPON'].fillna(0)

In [4]:
#group by order_date to get the max order for a date
count_per_date = pd.DataFrame(df_zamow.groupby('order_date').size(), columns = ['count'])

In [5]:
#sort the value from the biggest => we can use also max here on column
count_per_date = count_per_date.sort_values(by = 'count', ascending = False)

In [6]:
#return the date with the biggest orders (answer 1)
count_per_date.index[0]

'14.08.2019'

In [7]:
#retrieve kupon column whre different than 0
kupon = df_zamow[df_zamow['KUPON'] != 0]

In [8]:
kupon.head()

,order_id,client_id,order_date,KUPON
3,1896544,123641536,05.05.2019,15%
5,2143538,563294140,08.04.2019,40%
23,4932443,314452019,31.03.2019,20%
25,5231560,649526055,19.07.2019,5%
26,6074529,849773609,08.07.2019,30%


In [9]:
#check the client by their unique occurence in case someone made 2 orders (answer 2)
len(kupon.client_id.unique())

998

In [10]:
#create a DT with counting client id occurence in orders
num_order_client = pd.DataFrame(df_zamow.groupby('client_id').size(), columns = ['count'])

In [11]:
num_order_client.head()

,count
client_id,
107893,1
165795,1
288561,1
554772,1
713722,1


In [12]:
#order the DT from the biggest (can apply also max to get the results)
num_order_client = num_order_client.sort_values(by = 'count', ascending = False)

In [13]:
num_order_client.head()

,count
client_id,
723755074,3
405195042,2
3133665,2
724232743,2
723796610,2


In [14]:
#retrieve the client with more than 1 orders (answer 3)
len(num_order_client[num_order_client['count'] > 1])

22

In [15]:
#check in wyslane DT
#fill NAN with 0 in column item_id_1, item_id_2, item_id_3
df_wyslane['item_id_1'] = df_wyslane['item_id_1'].fillna(0)
df_wyslane['item_id_2'] = df_wyslane['item_id_2'].fillna(0)
df_wyslane['item_id_3'] = df_wyslane['item_id_3'].fillna(0)


In [16]:
#convert columns to numeric
df_wyslane['item_id_1'] = pd.to_numeric(df_wyslane['item_id_1'], downcast = 'integer')
df_wyslane['item_id_2'] = pd.to_numeric(df_wyslane['item_id_2'], downcast = 'integer')
df_wyslane['item_id_3'] = pd.to_numeric(df_wyslane['item_id_3'], downcast = 'integer')

In [17]:
#retrive data from item_id_1 and group by their number to get the occurence of each item
num_item_count_1 = pd.DataFrame(df_wyslane.groupby(['item_id_1'])
                              .size(), columns = ['count_1'])

In [18]:
#retrive data from item_id_2 and group by their number to get the occurence of each item
num_item_count_2 = pd.DataFrame(df_wyslane.groupby(['item_id_2'])
                              .size(), columns = ['count_2'])

In [19]:
#retrive data from item_id_3 and group by their number to get the occurence of each item
num_item_count_3 = pd.DataFrame(df_wyslane.groupby(['item_id_3'])
                              .size(), columns = ['count_3'])

In [20]:
#reset the index for better understanding in merging
num_item_count_1 = num_item_count_1.reset_index()
num_item_count_2 = num_item_count_2.reset_index()
num_item_count_3 = num_item_count_3.reset_index()

In [21]:
#drop rows where order_id = 0
num_item_count_1 = num_item_count_1[num_item_count_1['item_id_1'] != 0]
num_item_count_2 = num_item_count_2[num_item_count_2['item_id_2'] != 0]
num_item_count_3 = num_item_count_3[num_item_count_3['item_id_3'] != 0]

In [22]:
#merge the first count and the second one
merge_items_count_1 = pd.merge(num_item_count_1,num_item_count_2, how = 'outer', 
                                        right_on = 'item_id_2', left_on = 'item_id_1')

In [23]:
#merge the first 2 counts and the third one
merge_items_count_2 = pd.merge(merge_items_count_1, num_item_count_3, how = 'outer', 
                             right_on = 'item_id_3', left_on = 'item_id_2')

In [24]:
merge_items_count_2

,item_id_1,count_1,item_id_2,count_2,item_id_3,count_3
0,52730,38,52730,7,52730.0,2.0
1,152614,39,152614,14,152614.0,3.0
2,331250,37,331250,10,331250.0,2.0
3,534353,22,534353,8,534353.0,7.0
4,552870,30,552870,12,552870.0,4.0
...,...,...,...,...,...,...
105,9780720,39,9780720,14,9780720.0,3.0
106,9813052,29,9813052,14,9813052.0,7.0
107,9873648,38,9873648,11,9873648.0,5.0
108,9897093,40,9897093,9,9897093.0,3.0


In [25]:
#create a column total to get the value of the biggest product
merge_items_count_2['total'] = merge_items_count_2['count_1'] + merge_items_count_2['count_2'] + merge_items_count_2['count_3']

In [26]:
#return the item id of the popular products (answer 4.1)
(merge_items_count_2[merge_items_count_2['total'] == merge_items_count_2['total'].max()]).iloc[0, 0]

4355722

In [27]:
#check the kupon DT
kupon

,order_id,client_id,order_date,KUPON
3,1896544,123641536,05.05.2019,15%
5,2143538,563294140,08.04.2019,40%
23,4932443,314452019,31.03.2019,20%
25,5231560,649526055,19.07.2019,5%
26,6074529,849773609,08.07.2019,30%
...,...,...,...,...
4998,993806672,733932462,14.08.2019,43%
5005,994770134,260389043,18.05.2019,45%
5013,996541103,989867713,25.08.2019,29%
5014,997108649,182839695,31.03.2019,22%


In [28]:
#check wyslane DT
df_wyslane

,order_id,item_id_1,item_id_2,item_id_3
0,314631735,2655227,9051607,0
1,802224752,1984910,4286301,0
2,716057036,2669985,1052038,0
3,125731133,5966397,0,0
4,12070864,6788106,9643817,0
...,...,...,...,...
3662,552069972,4738768,684194,4169115
3663,408612183,8250741,1005647,0
3664,725171229,152614,0,0
3665,26224680,1971413,0,0


In [29]:
#merge the 2 DT to retrieve the number client (left because we will get the client with coupon only)
client_kupon = pd.merge(kupon, df_wyslane, how = 'left', on ='order_id')

In [30]:
#check the client_kupon DT
client_kupon

,order_id,client_id,order_date,KUPON,item_id_1,item_id_2,item_id_3
0,1896544,123641536,05.05.2019,15%,NaN,NaN,NaN
1,2143538,563294140,08.04.2019,40%,5801454.0,2077835.0,0.0
2,4932443,314452019,31.03.2019,20%,6960602.0,0.0,0.0
3,5231560,649526055,19.07.2019,5%,7642810.0,0.0,0.0
4,6074529,849773609,08.07.2019,30%,2773385.0,0.0,0.0
...,...,...,...,...,...,...,...
993,993806672,733932462,14.08.2019,43%,8266482.0,0.0,0.0
994,994770134,260389043,18.05.2019,45%,NaN,NaN,NaN
995,996541103,989867713,25.08.2019,29%,6615435.0,0.0,0.0
996,997108649,182839695,31.03.2019,22%,NaN,NaN,NaN


In [31]:
#retrive the clients which only bought the popular product 
number_client = client_kupon[(client_kupon['item_id_1'] == 4355722) | (client_kupon['item_id_2'] == 4355722)
                            | (client_kupon['item_id_3'] == 4355722)]

In [32]:
#check the len of the DT to get the correct number (answer 4.1)
len(number_client)

11

In [33]:
#retrieve kupon for item_id_1
coupon_item_count_1 = pd.DataFrame(client_kupon.groupby(['item_id_1'])
                              .size(), columns = ['count_1'])

In [34]:
#retrieve kupon for item_id_2
coupon_item_count_2 = pd.DataFrame(client_kupon.groupby(['item_id_2'])
                              .size(), columns = ['count_2'])

In [35]:
#retrieve kupon for item_id_1
coupon_item_count_3 = pd.DataFrame(client_kupon.groupby(['item_id_3'])
                              .size(), columns = ['count_3'])

In [36]:
#reset the index for better understanding on merging
coupon_item_count_1 = coupon_item_count_1.reset_index()
coupon_item_count_2 = coupon_item_count_2.reset_index()
coupon_item_count_3 = coupon_item_count_3.reset_index()

In [37]:
#drop rows where order_id = 0
coupon_item_count_1 = coupon_item_count_1[coupon_item_count_1['item_id_1'] != 0]
coupon_item_count_2 = coupon_item_count_2[coupon_item_count_2['item_id_2'] != 0]
coupon_item_count_3 = coupon_item_count_3[coupon_item_count_3['item_id_3'] != 0]

In [38]:
#merge first and second kupon DT
merge_kupon_count_1 = pd.merge(coupon_item_count_1,coupon_item_count_2, how = 'outer', 
                                        right_on = 'item_id_2', left_on = 'item_id_1')

In [39]:
#merge the first 2 datasets and third kupon DT
merge_kupon_count_2 = pd.merge(merge_kupon_count_1, coupon_item_count_3, how = 'outer', 
                              right_on = 'item_id_3', left_on = 'item_id_2')

In [40]:
merge_kupon_count_2.head()

,item_id_1,count_1,item_id_2,count_2,item_id_3,count_3
0,52730.0,11.0,52730.0,1.0,NaN,NaN
1,152614.0,5.0,152614.0,3.0,152614.0,1.0
2,331250.0,6.0,331250.0,1.0,NaN,NaN
3,534353.0,3.0,534353.0,3.0,NaN,NaN
4,552870.0,10.0,552870.0,5.0,552870.0,1.0


In [41]:
#let set NaN values to 0
merge_kupon_count_2 = merge_kupon_count_2.fillna(0)

In [42]:
merge_kupon_count_2.head()

,item_id_1,count_1,item_id_2,count_2,item_id_3,count_3
0,52730.0,11.0,52730.0,1.0,0.0,0.0
1,152614.0,5.0,152614.0,3.0,152614.0,1.0
2,331250.0,6.0,331250.0,1.0,0.0,0.0
3,534353.0,3.0,534353.0,3.0,0.0,0.0
4,552870.0,10.0,552870.0,5.0,552870.0,1.0


In [43]:
#create the total column
merge_kupon_count_2['total'] = merge_kupon_count_2['count_1'] + merge_kupon_count_2['count_2'] + merge_kupon_count_2['count_3']

In [44]:
merge_kupon_count_2

,item_id_1,count_1,item_id_2,count_2,item_id_3,count_3,total
0,52730.0,11.0,52730.0,1.0,0.0,0.0,12.0
1,152614.0,5.0,152614.0,3.0,152614.0,1.0,9.0
2,331250.0,6.0,331250.0,1.0,0.0,0.0,7.0
3,534353.0,3.0,534353.0,3.0,0.0,0.0,6.0
4,552870.0,10.0,552870.0,5.0,552870.0,1.0,16.0
...,...,...,...,...,...,...,...
106,9813052.0,4.0,9813052.0,1.0,9813052.0,1.0,6.0
107,9873648.0,12.0,9873648.0,3.0,9873648.0,2.0,17.0
108,9897093.0,9.0,9897093.0,1.0,9897093.0,1.0,11.0
109,9989669.0,5.0,9989669.0,2.0,9989669.0,1.0,8.0


In [45]:
#return the product bought with kupon most of times (answer 5)
int((merge_kupon_count_2[merge_kupon_count_2['total'] == merge_kupon_count_2['total'].max()]).iloc[0, 0])

1554376

In [46]:
#check wyslane DT
df_wyslane.head()

,order_id,item_id_1,item_id_2,item_id_3
0,314631735,2655227,9051607,0
1,802224752,1984910,4286301,0
2,716057036,2669985,1052038,0
3,125731133,5966397,0,0
4,12070864,6788106,9643817,0


In [47]:
#retrieve the order by item_id solds
wyslane_it_1 = df_wyslane[['order_id', 'item_id_1']]
wyslane_it_2 = df_wyslane[['order_id', 'item_id_2']]
wyslane_it_3 = df_wyslane[['order_id', 'item_id_3']]

In [48]:
wyslane_it_1 = wyslane_it_1[wyslane_it_1['item_id_1'] != 0]
wyslane_it_2 = wyslane_it_2[wyslane_it_2['item_id_2'] != 0]
wyslane_it_3 = wyslane_it_3[wyslane_it_3['item_id_3'] != 0]

In [49]:
wyslane_it_3

,order_id,item_id_3
10,59848447,2655227
16,137806540,8948103
87,852730828,2502612
93,354494585,7706124
101,152428796,2247054
...,...,...
3600,104230078,2350739
3643,872365392,2565383
3650,379311060,9897093
3662,552069972,4169115


In [50]:
#group by item_id in order to count the number sold
wyslane_it_1 = pd.DataFrame(wyslane_it_1.groupby(['item_id_1'])
                              .size(), columns = ['count_1'])
wyslane_it_2 = pd.DataFrame(wyslane_it_2.groupby(['item_id_2'])
                              .size(), columns = ['count_2'])
wyslane_it_3 = pd.DataFrame(wyslane_it_3.groupby(['item_id_3'])
                              .size(), columns = ['count_3'])

In [51]:
wyslane_it_3

,count_3
item_id_3,
52730,2
152614,3
331250,2
534353,7
552870,4
...,...
9780720,3
9813052,7
9873648,5


In [52]:
#reset index
wyslane_it_1 = wyslane_it_1.reset_index()
wyslane_it_2 = wyslane_it_2.reset_index()
wyslane_it_3 = wyslane_it_3.reset_index()

In [53]:
#merge each DT of item_id with cena DT
merge_wyslane_item_1 = pd.merge(wyslane_it_1, df_cena, how = 'left', right_on = 'item_id',
                        left_on = 'item_id_1')
merge_wyslane_item_2 = pd.merge(wyslane_it_2, df_cena, how = 'left', right_on = 'item_id',
                        left_on = 'item_id_2')
merge_wyslane_item_3 = pd.merge(wyslane_it_3, df_cena, how = 'left', right_on = 'item_id',
                        left_on = 'item_id_3')

In [54]:
merge_wyslane_item_3

,item_id_3,count_3,item_id,base_price
0,52730,2,52730,205
1,152614,3,152614,145
2,331250,2,331250,183
3,534353,7,534353,78
4,552870,4,552870,242
...,...,...,...,...
101,9780720,3,9780720,132
102,9813052,7,9813052,211
103,9873648,5,9873648,153
104,9897093,3,9897093,202


In [55]:
#calculate the total price for each item_id in each DT
merge_wyslane_item_1['total'] = merge_wyslane_item_1['count_1'] * merge_wyslane_item_1['base_price']
merge_wyslane_item_2['total'] = merge_wyslane_item_2['count_2'] * merge_wyslane_item_2['base_price']
merge_wyslane_item_3['total'] = merge_wyslane_item_3['count_3'] * merge_wyslane_item_3['base_price']


In [56]:
#compute the total of all DT
total = merge_wyslane_item_1['total'].sum() + merge_wyslane_item_2['total'].sum() + merge_wyslane_item_3['total'].sum()

In [57]:
total #answer 6

838391

#### ZADANIE 2 

###### W pliku predykcja.csv znajdują się dane o liczbie zamówień w okresie 01.01.2018 – 01.01.2020.  w kolumnie wyprzedaż oznaczono okresy wyprzedaży. Użyj wybranego modelu, żeby zaprognozować sprzedaż w styczniu i lutym 2020. 
    Jak zmieniłaby się Twoja predykcja, gdybyś wiedział(a), że w dniach 01.02.2020 – 07.02.2020 planowana jest wyprzedaż?
    Jako rozwiązanie proszę o przesłanie szacowanej liczby zamówień za styczeń i luty 2020 przy założeniu wyprzedaży w lutym i bez wyprzedaży oraz informacje o krokach analizy i wybranym modelu. 

In [58]:
#lod the dataset
df_predykcja  = pd.read_csv('predykcja.csv', sep =';')

In [59]:
#the number of columns in the dataset are 731
df_predykcja = df_predykcja[:731]

In [60]:
df_predykcja

,DATA,zamowienia,wyprzedaż
0,01.01.2018,100.0,NaN
1,02.01.2018,94.0,NaN
2,03.01.2018,118.0,1.0
3,04.01.2018,114.0,1.0
4,05.01.2018,122.0,1.0
...,...,...,...
726,28.12.2019,260.0,NaN
727,29.12.2019,257.0,NaN
728,30.12.2019,257.0,NaN
729,31.12.2019,254.0,NaN


In [61]:
df_predykcja.columns

Index(['DATA', 'zamowienia', 'wyprzedaż'], dtype='object')

In [62]:
#fill NaN in wyprzedaz with 0
df_predykcja['wyprzedaż'] = df_predykcja.wyprzedaż.fillna(0)

In [63]:
#convert data from zamowienia and wyprzedaz to integers
df_predykcja['zamowienia'] = pd.to_numeric(df_predykcja['zamowienia'], downcast = 'integer')
df_predykcja['wyprzedaż'] = pd.to_numeric(df_predykcja['wyprzedaż'], downcast = 'integer')

In [64]:
#convert DATA column to datetime type
df_predykcja['DATA'] = df_predykcja['DATA'].astype('datetime64[ns]')

In [65]:
#checking the info of our dataframe
df_predykcja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731 entries, 0 to 730
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   DATA        731 non-null    datetime64[ns]
 1   zamowienia  731 non-null    int16         
 2   wyprzedaż   731 non-null    int8          
dtypes: datetime64[ns](1), int16(1), int8(1)
memory usage: 8.0 KB


In [66]:
df_predykcja.head()

,DATA,zamowienia,wyprzedaż
0,2018-01-01,100,0
1,2018-02-01,94,0
2,2018-03-01,118,1
3,2018-04-01,114,1
4,2018-05-01,122,1


In [67]:
#create columns for day, month and year
df_predykcja['day'] = df_predykcja['DATA'].dt.day
df_predykcja['month'] = df_predykcja['DATA'].dt.month
df_predykcja['year'] = df_predykcja['DATA'].dt.year

In [68]:
df_predykcja.head()

,DATA,zamowienia,wyprzedaż,day,month,year
0,2018-01-01,100,0,1,1,2018
1,2018-02-01,94,0,1,2,2018
2,2018-03-01,118,1,1,3,2018
3,2018-04-01,114,1,1,4,2018
4,2018-05-01,122,1,1,5,2018


In [69]:
#create a dataframe with features
X = df_predykcja[['day', 'month', 'year', 'wyprzedaż']]
#create a series with label or targeted values
y = df_predykcja['zamowienia']

In [70]:
#checking the shapes
print(X.shape)
print(y.shape)

(731, 4)
(731,)


In [71]:
#import librairies for training, testing and modelling
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [72]:
#split the date between train and test data for modeling and testing
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [73]:
#create a model and train the data
linreg = LinearRegression().fit(X_train, y_train)

In [74]:
#predict the values for X_test
y_pred = linreg.predict(X_test)

In [75]:
#checking the metrics
from sklearn import metrics

#check mean absolute error
print('Mean absolute error', metrics.mean_absolute_error(y_test, y_pred))
#check the mean squared error
print('Mean squared error', metrics.mean_squared_error(y_test, y_pred))
#check the roor mean squared error
print('Root mean squared error', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
#check coefficients
print('Coefficients ', linreg.coef_)
#check intercept
print('Intercept value', linreg.intercept_)


Mean absolute error 33.34290571227904
Mean squared error 1668.792941143455
Root mean squared error 40.85086218359969
Coefficients  [ 0.70016632  3.42622526 40.011239   16.97272179]
Intercept value -80633.00237237914


## Prediction from jan and feb

In [76]:
from datetime import date, timedelta

#create the date for prediction
start_date = date(2020, 1, 2)
end_date = date(2020, 3, 1)

list_days = pd.date_range(start_date, end_date - timedelta(days =1), freq = 'd')

In [77]:
list_days

DatetimeIndex(['2020-01-02', '2020-01-03', '2020-01-04', '2020-01-05',
               '2020-01-06', '2020-01-07', '2020-01-08', '2020-01-09',
               '2020-01-10', '2020-01-11', '2020-01-12', '2020-01-13',
               '2020-01-14', '2020-01-15', '2020-01-16', '2020-01-17',
               '2020-01-18', '2020-01-19', '2020-01-20', '2020-01-21',
               '2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25',
               '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29',
               '2020-01-30', '2020-01-31', '2020-02-01', '2020-02-02',
               '2020-02-03', '2020-02-04', '2020-02-05', '2020-02-06',
               '2020-02-07', '2020-02-08', '2020-02-09', '2020-02-10',
               '2020-02-11', '2020-02-12', '2020-02-13', '2020-02-14',
               '2020-02-15', '2020-02-16', '2020-02-17', '2020-02-18',
               '2020-02-19', '2020-02-20', '2020-02-21', '2020-02-22',
               '2020-02-23', '2020-02-24', '2020-02-25', '2020-02-26',
      

In [78]:
#create a dateframe with the newly created date
df_y_pred = pd.DataFrame(list_days, columns = ['DATA'])

In [79]:
#created a column named wyprzedaz in the dataframe without sales filled
df_y_pred['wyprzedaż'] = 0

In [80]:
#create a copy of dataframe with sales now from 2/2 to 7/2
df_y_pred_with_sales = df_y_pred.copy()

In [81]:
#fill sales from 2/3 to 7/2
df_y_pred_with_sales.loc[(df_y_pred_with_sales['DATA'] > '2020-02-01')
                         & (df_y_pred_with_sales['DATA'] < '2020-02-08'), 'wyprzedaż'] = 1

In [82]:
#create day, month and year for dataframe without sales from 2/2 to 7/2
df_y_pred['day'] = df_y_pred['DATA'].dt.day
df_y_pred['month'] = df_y_pred['DATA'].dt.month
df_y_pred['year'] = df_y_pred['DATA'].dt.year
#create day, month and year for dataframe with sales from 2/2 to 7/2
df_y_pred_with_sales['day'] = df_y_pred_with_sales['DATA'].dt.day
df_y_pred_with_sales['month'] = df_y_pred_with_sales['DATA'].dt.month
df_y_pred_with_sales['year'] = df_y_pred_with_sales['DATA'].dt.year

In [83]:
#create testing data for our 2 dataframes
df_y_pred = df_y_pred[['day','month', 'year', 'wyprzedaż']]
df_y_pred_with_sales = df_y_pred_with_sales[[ 'day','month', 'year', 'wyprzedaż']] 

In [84]:
df_y_pred

,day,month,year,wyprzedaż
0,2,1,2020,0
1,3,1,2020,0
2,4,1,2020,0
3,5,1,2020,0
4,6,1,2020,0
5,7,1,2020,0
6,8,1,2020,0
7,9,1,2020,0
8,10,1,2020,0
9,11,1,2020,0


In [85]:
df_y_pred_with_sales

,day,month,year,wyprzedaż
0,2,1,2020,0
1,3,1,2020,0
2,4,1,2020,0
3,5,1,2020,0
4,6,1,2020,0
5,7,1,2020,0
6,8,1,2020,0
7,9,1,2020,0
8,10,1,2020,0
9,11,1,2020,0


In [86]:
#train the data with the all dataset of predykcja file
linreg = LinearRegression().fit(X, y)

In [87]:
#predict zamowienia without sales in february
y_pred_without_sales = linreg.predict(df_y_pred)

In [88]:
#predict zamowienia with sales in february
y_pred_with_sales = linreg.predict(df_y_pred_with_sales)

In [89]:
y_pred_without_sales

array([195.70028618, 196.56477742, 197.42926866, 198.2937599 ,
       199.15825114, 200.02274238, 200.88723362, 201.75172485,
       202.61621609, 203.48070733, 204.34519857, 205.20968981,
       206.07418105, 206.93867229, 207.80316353, 208.66765477,
       209.53214601, 210.39663724, 211.26112848, 212.12561972,
       212.99011096, 213.8546022 , 214.71909344, 215.58358468,
       216.44807592, 217.31256716, 218.1770584 , 219.04154964,
       219.90604087, 220.77053211, 198.48526297, 199.34975421,
       200.21424545, 201.07873669, 201.94322793, 202.80771917,
       203.67221041, 204.53670165, 205.40119289, 206.26568413,
       207.13017536, 207.9946666 , 208.85915784, 209.72364908,
       210.58814032, 211.45263156, 212.3171228 , 213.18161404,
       214.04610528, 214.91059652, 215.77508776, 216.63957899,
       217.50407023, 218.36856147, 219.23305271, 220.09754395,
       220.96203519, 221.82652643, 222.69101767])

In [90]:
sales_1_1 = df_predykcja.loc[730][1]

In [91]:
zamowienia_jan_fev = sales_1_1 + y_pred_without_sales.sum()

In [92]:
print('Predykcja bez planowana wyprzedaż w dniach  01.02.2020 – 07.02.2020: ', zamowienia_jan_fev)

Predykcja bez planowana wyprzedaż w dniach  01.02.2020 – 07.02.2020:  12605.11834373021


In [93]:
y_pred_with_sales

array([195.70028618, 196.56477742, 197.42926866, 198.2937599 ,
       199.15825114, 200.02274238, 200.88723362, 201.75172485,
       202.61621609, 203.48070733, 204.34519857, 205.20968981,
       206.07418105, 206.93867229, 207.80316353, 208.66765477,
       209.53214601, 210.39663724, 211.26112848, 212.12561972,
       212.99011096, 213.8546022 , 214.71909344, 215.58358468,
       216.44807592, 217.31256716, 218.1770584 , 219.04154964,
       219.90604087, 220.77053211, 198.48526297, 217.74734625,
       218.61183749, 219.47632873, 220.34081997, 221.20531121,
       222.06980245, 204.53670165, 205.40119289, 206.26568413,
       207.13017536, 207.9946666 , 208.85915784, 209.72364908,
       210.58814032, 211.45263156, 212.3171228 , 213.18161404,
       214.04610528, 214.91059652, 215.77508776, 216.63957899,
       217.50407023, 218.36856147, 219.23305271, 220.09754395,
       220.96203519, 221.82652643, 222.69101767])

In [94]:
zamowienia_jan_fev_sales = sales_1_1 + y_pred_with_sales.sum()

In [95]:
print('Predykcja z planowana wyprzedaż w dniach  01.02.2020 – 07.02.2020: ', zamowienia_jan_fev_sales)

Predykcja z planowana wyprzedaż w dniach  01.02.2020 – 07.02.2020:  12715.50389597124


In [96]:
'''import matplotlib.pyplot as plt

plt.figure()
plt.plot(df_predykcja['zamowienia'])''';